In [ ]:
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import requests
from zipfile import ZipFile
from io import BytesIO

# Définir les URL des ensembles de données et les chemins locaux
datasets = {
    "SGD": {
        "url": "https://github.com/google-research-datasets/dstc8-schema-guided-dialogue/archive/refs/heads/master.zip",
        "path": "data/SGD/"
    },
    "MultiWOZ": {
        "url": "https://github.com/budzianowski/multiwoz/archive/refs/heads/master.zip",
        "path": "data/MultiWOZ/"
    },
    "ABCD": {
        "url": "https://github.com/asappresearch/abcd/raw/master/data/abcd_v1.1.json.gz",
        "path": "data/ABCD/"
    },
    "BiToD": {
        "url": "https://github.com/HLTCHKUST/BiToD/archive/refs/heads/main.zip",
        "path": "data/BiToD/"
    },
    "SMCalFlow": {
        "url": "https://github.com/microsoft/task_oriented_dialogue_as_dataflow_synthesis/archive/refs/heads/main.zip",
        "path": "data/SMCalFlow/"
    },
    "TreeDST": {
        "url": "https://github.com/microsoft/task_oriented_dialogue_as_dataflow_synthesis/archive/refs/heads/main.zip",
        "path": "data/TreeDST/"
    }
}

# Fonction pour télécharger et extraire les ensembles de données
def download_and_extract(dataset_name, dataset_info):
    if not os.path.exists(dataset_info["path"]):
        os.makedirs(dataset_info["path"])
        print(f"Téléchargement de {dataset_name}...")
        response = requests.get(dataset_info["url"])
        with ZipFile(BytesIO(response.content)) as zip_ref:
            zip_ref.extractall(dataset_info["path"])
        print(f"{dataset_name} téléchargé et extrait.")
    else:
        print(f"{dataset_name} existe déjà. Téléchargement ignoré.")

# Télécharger les ensembles de données
for name, info in datasets.items():
    download_and_extract(name, info)


In [ ]:
import os
import gzip
import shutil
from zipfile import ZipFile

def check_and_unzip(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            # Handling .zip files
            if file.endswith('.zip'):
                extract_path = os.path.splitext(file_path)[0]
                if not os.path.exists(extract_path):
                    print(f"Extraction de {file_path}...")
                    with ZipFile(file_path, 'r') as zip_ref:
                        zip_ref.extractall(extract_path)
                    print(f"{file_path} extrait.")
                else:
                    print(f"{extract_path} existe déjà. Extraction ignorée.")
            # Handling .gz files
            elif file.endswith('.gz'):
                extract_path = os.path.splitext(file_path)[0]  # Remove the .gz extension
                if not os.path.exists(extract_path):
                    print(f"Extraction de {file_path}...")
                    with gzip.open(file_path, 'rb') as f_in:
                        with open(extract_path, 'wb') as f_out:
                            shutil.copyfileobj(f_in, f_out)
                    print(f"{file_path} extrait.")
                else:
                    print(f"{extract_path} existe déjà. Extraction ignorée.")

# Call the function with your directory
check_and_unzip('data/ABCD')


In [ ]:
import os

def get_folder_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

def print_folder_sizes(base_directory):
    for root, dirs, files in os.walk(base_directory):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            size = get_folder_size(dir_path)
            print(f"Folder: {dir_path}, Size: {size / (1024 ** 2):.2f} MB")  # Convert bytes to MB

# Change 'data' to your base folder name if needed
print_folder_sizes('./data')


In [3]:
datasets = {
    "SGD": {
        "url": "https://github.com/google-research-datasets/dstc8-schema-guided-dialogue/archive/refs/heads/master.zip",
        "path": "data/SGD/"
    },
    "MultiWOZ": {
        "url": "https://github.com/budzianowski/multiwoz/archive/refs/heads/master.zip",
        "path": "data/MultiWOZ/"
    },
    "ABCD": {
        "url": "https://github.com/asappresearch/abcd/raw/master/data/abcd_v1.1.json.gz",
        "path": "data/ABCD/"
    },
    "BiToD": {
        "url": "https://github.com/HLTCHKUST/BiToD/archive/refs/heads/main.zip",
        "path": "data/BiToD/"
    },
    "SMCalFlow": {
        "url": "https://github.com/microsoft/task_oriented_dialogue_as_dataflow_synthesis/archive/refs/heads/main.zip",
        "path": "data/SMCalFlow/"
    },
    "TreeDST": {
        "url": "https://github.com/microsoft/task_oriented_dialogue_as_dataflow_synthesis/archive/refs/heads/main.zip",
        "path": "data/TreeDST/"
    }
}

# Function to load JSON files from a directory
def load_json_files(directory):
    data = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    data.append(json.load(f))
    return data

# Function to explore dataset
def explore_dataset(dataset_name, dataset_path):
    print(f"\nExploring {dataset_name}...")

    # Load data
    data = load_json_files(dataset_path)

    # Example: Analyze dialogue lengths
    dialogue_lengths = [len(dialogue['turns']) for dialogue in data]
    print(f"Average dialogue length: {sum(dialogue_lengths) / len(dialogue_lengths):.2f} turns")

    # Plot dialogue length distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(dialogue_lengths, bins=30, kde=True)
    plt.title(f"{dataset_name} Dialogue Length Distribution")
    plt.xlabel("Number of Turns")
    plt.ylabel("Frequency")
    plt.show()

    # Example: Analyze intents (if available)
    intents = []
    for dialogue in data:
        for turn in dialogue['turns']:
            if 'intent' in turn:
                intents.append(turn['intent'])

    if intents:
        intent_counts = Counter(intents)
        print(f"Top 5 intents in {dataset_name}: {intent_counts.most_common(5)}")

        # Plot intent distribution
        plt.figure(figsize=(12, 8))
        sns.barplot(x=[count for _, count in intent_counts.most_common(10)],
                    y=[intent for intent, _ in intent_counts.most_common(10)])
        plt.title(f"Top 10 Intents in {dataset_name}")
        plt.xlabel("Frequency")
        plt.ylabel("Intent")
        plt.show()

    # Additional analyses can be added here (e.g., slot distributions, confusion matrices)

# Explore each dataset
for name, info in datasets.items():
    explore_dataset(name, info["path"])


Exploring SGD...


NameError: name 'json' is not defined